In [1]:
from libraries import *
from parameters import *
import statsmodels.api as sm

In [2]:
os.getcwd()
os.chdir(projectDir)

In [3]:
adata = sc.read('outputs/anndata/adata-hash-features_singlets_SingleKO_05242020.h5ad')
adata.X.shape

(341664, 13811)

In [22]:
adata.uns['Control_guides'] = [ x for x in adata.uns['feature_barcode_names'] if (x.startswith("ONE_NONGENE_SITE") or x.startswith("NO_TARGET"))]
controlGM = adata.obs[adata.uns['Control_guides']]
controlGM.shape

(341664, 330)

In [23]:
controlGM =controlGM[controlGM.sum(axis=1) == 1]
adataControl = adata[controlGM.index,]
adataControl.shape

(44074, 13811)

In [24]:
adataControl.obs['Control_guide'] = adataControl.obs[adataControl.uns['Control_guides']].idxmax(axis=1)
adataControl.obs['Control_guide']

/tmp/ipykernel_42943/1481002648.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adataControl.obs['Control_guide'] = adataControl.obs[adataControl.uns['Control_guides']].idxmax(axis=1)


AAACGCTAGTTGTAGA-1-E3-1     ONE_NONGENE_SITE_192
AAAGGATCAGGTTTAC-1-E3-1             NO_TARGET_60
AAAGGATTCAGACAAA-1-E3-1     ONE_NONGENE_SITE_203
AAAGGGCAGACGGTTG-1-E3-1            NO_TARGET_148
AAAGGGCAGAGATCGC-1-E3-1            NO_TARGET_101
                                    ...         
TTTGATCCAAACGAGC-1-E4-24    ONE_NONGENE_SITE_243
TTTGATCCATGACAAA-1-E4-24    ONE_NONGENE_SITE_287
TTTGATCTCAGCTGTA-1-E4-24            NO_TARGET_81
TTTGGAGTCCGTAGGC-1-E4-24            NO_TARGET_51
TTTGGTTAGTTTAGGA-1-E4-24            NO_TARGET_79
Name: Control_guide, Length: 44074, dtype: object

In [26]:
sc.tl.pca(adataControl, use_highly_variable=True,n_comps=100, svd_solver='arpack')

computing PCA
    on highly variable genes
    with n_comps=100
    finished (0:00:07)


In [27]:
expressionMatrix = adataControl.obsm['X_pca']
guideMatrix = adataControl.obs[adataControl.uns['Control_guides']]

In [28]:
pvalDF = pd.DataFrame()
coefDF = pd.DataFrame()
    
for j in range(0,100):
    print(j)
    est = sm.OLS(np.array(expressionMatrix)[:,j], np.array(guideMatrix))
    est2 = est.fit().summary2().tables[1]
    pvalDF[str(j)] = est2['P>|t|']
    coefDF[str(j)] = est2['Coef.']
    
# pvalDF.to_csv("ControlGuides_pvalues.csv")   
# coefDF.to_csv("ControlGuides_coefs.csv")   
 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [29]:
adataControl.obsm['X_pca'].shape

(44074, 100)

In [30]:
coefDF.index = adataControl.uns['Control_guides']

In [41]:
coefDF.shape

(330, 100)

In [31]:
from sklearn.ensemble import IsolationForest

iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(coefDF)
k = pd.Series(coefDF.index[yhat == -1,])

/home/eraslab1/miniconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [32]:
from sklearn.covariance import EllipticEnvelope
ee = EllipticEnvelope(contamination=0.1)
yhat = ee.fit_predict(coefDF)

k = k.append(pd.Series(coefDF.index[yhat == -1,]))

/tmp/ipykernel_42943/3453177345.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  k = k.append(pd.Series(coefDF.index[yhat == -1,]))


In [33]:
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor()
yhat = lof.fit_predict(coefDF)
k = k.append(pd.Series(coefDF.index[yhat == -1,]))

/tmp/ipykernel_42943/901771125.py:4: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  k = k.append(pd.Series(coefDF.index[yhat == -1,]))


In [34]:
from sklearn.svm import OneClassSVM
ee = OneClassSVM(nu=0.01)
yhat = ee.fit_predict(coefDF)
k = k.append(pd.Series(coefDF.index[yhat == -1,]))

/tmp/ipykernel_42943/11974079.py:4: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  k = k.append(pd.Series(coefDF.index[yhat == -1,]))


In [35]:
allSel = pd.DataFrame(k.value_counts())
allSel.columns = ["dene"]

In [37]:
allSel = allSel.loc[ allSel.dene > 2,]
allSel['OutlierGuides'] = allSel.index

/tmp/ipykernel_42943/2423422964.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allSel['OutlierGuides'] = allSel.index


In [39]:
allSel = allSel.sort_values(by=['OutlierGuides'])

In [40]:
allSel

,dene,OutlierGuides
NO_TARGET_104,3,NO_TARGET_104
NO_TARGET_135,4,NO_TARGET_135
NO_TARGET_149,3,NO_TARGET_149
NO_TARGET_151,4,NO_TARGET_151
NO_TARGET_17,3,NO_TARGET_17
NO_TARGET_56,3,NO_TARGET_56
NO_TARGET_58,4,NO_TARGET_58
NO_TARGET_64,4,NO_TARGET_64
NO_TARGET_87,3,NO_TARGET_87
ONE_NONGENE_SITE_174,3,ONE_NONGENE_SITE_174


In [ ]:
#allSel.loc[:,"OutlierGuides"].to_csv("BadControlGuides_v2.csv", index=False)